In [4]:
from langchain_groq import ChatGroq

In [ ]:
llm= ChatGroq(
temperature=0,
groq_api_key='--',
model_name="llama-3.3-70b-versatile"
)

response=llm.invoke("Who is Modi (in 50 words)....")
print(response.content)

Narendra Modi is the 14th Prime Minister of India, serving since 2014. A member of the Bharatiya Janata Party (BJP), he is known for his nationalist and economic development policies, and is a prominent global leader, promoting India's interests and growth.


**WebAseloader** is used in **LangChain** to scrape and load web pages as documents for **LLMs, vector databases, and NLP applications** efficiently.

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader= WebBaseLoader("https://careers.nike.com/analyst-finance-gc/job/R-45280")
page_data = loader.load().pop().page_content
print(page_data)






















Analyst, Finance GC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Language


In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
"""
### SCRAPED TEXT FROM WEBSITE:
{page_data}
### INSTRUCTION:
The scraped text is from the career's page of a website.
Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
Only return the valid JSON.
### VALID JSON (NO PREAMBLE):
"""
)

chain_extract=prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
{
  "role": "Analyst, Finance GC",
  "experience": "At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus",
  "skills": [
    "Microsoft Excel",
    "Outlook",
    "Word",
    "PowerPoint",
    "Power BI",
    "Tableau"
  ],
  "description": "This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks"
}
```


In [18]:
type(res.content)

str

In [22]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Analyst, Finance GC',
 'experience': 'At least 5 years’ experience of audit or finance analyst position in MNC, Media or Creative agency working experience is a plus',
 'skills': ['Microsoft Excel',
  'Outlook',
  'Word',
  'PowerPoint',
  'Power BI',
  'Tableau'],
 'description': 'This is a commercial FP&A role and key responsibility is to support Icon. Shanghai Studio Financial planning and analysis for Business team. Support monthly actual and forecast spending collection, validation, and variance analysis. Support system maintenance and development to facilitate Icon financial process. Support annual budgeting and 3 years strategic plan development. Prepare financial related review reports and deliverables. Cross function communication with Icon. Studio team and other finance team. Other ad hoc tasks'}

In [24]:
type(json_res)

dict

In [56]:
import pandas as pd

df=pd.read_csv(r"C:\Users\DELL\Gen Ai\Code Basics\ColdEmailGenerator\app\resource\my_portfolio.csv")
df.head(5)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [28]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [36]:
links=collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [38]:
job=json_res
job['skills']

['Microsoft Excel', 'Outlook', 'Word', 'PowerPoint', 'Power BI', 'Tableau']

In [58]:
link = collection.query(query_texts=[" ".join(job['skills'])]).get('metadatas')
link

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [60]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )
chain_email=prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)




Subject: Enhancing Financial Planning and Analysis with AtliQ's Expertise

Dear Hiring Manager,

I came across the job description for an Analyst, Finance GC, and I was impressed by the role's focus on financial planning and analysis. As a Business Development Executive at AtliQ, I believe our company can provide the necessary support to enhance your financial processes.

At AtliQ, we specialize in developing tailored solutions to facilitate seamless integration of business processes through automated tools. Our expertise in Microsoft Excel, Outlook, Word, PowerPoint, Power BI, and Tableau can help streamline your financial planning and analysis. We can assist in monthly actual and forecast spending collection, validation, and variance analysis, as well as system maintenance and development to facilitate your financial process.

Our team has experience in developing solutions that can help with annual budgeting and 3-year strategic plan development. We can also provide support in prepa

- streamlit run "C:\Users\DELL\Gen Ai\Code Basics\ColdEmailGenerator\app\main.py"


In [ ]:
import os

file_name="
